# 임베디드 AI 기말 과제 보고서

## Model

이 모델은 yolov5모델을 통하여 한자 필기체 데이터를 한-일 ~ 열-십까지 학습시켜서 인식하는 모델입니다.


각 필기체의 데이터는 글자당 50개지만 이 이미지를 변환하여 총 2000장의 이미지를 통해 학습하였으며, 새로운 이미지들은 하얀 바탕에 무작위적인 위치에 임의의 크기의 한자 필기체가 부유한 형태로 그 위에 음영을 추가한 데이터입니다.


학습에는 총 3시간이 소요되었으며, 모델의 이미지 내에서의 한자 인식률은 높은 편입니다. (마지막 에포크에서 metrics/mAP_0.5 = 99.175%,metrics/mAP_0.5:0.95 = 84.506%)


경량화 이전 모델은 실전에서는 적당한 크기의 글씨에 대해서만 인식을 온전히 하는 모습을 보였습니다. 다만, 물체와 다른 글자들을 한자로 오인하는 경우가 많았습니다. 


모델은 아무것도 감지 하지 않는 경우 fps(초당 프레임)이 9에서 머무르고 인식을 하는 경우는 보통 7fps에서 최저 6fps까지 내려가는 모습을 볼 수 있었습니다. 

## Pruning

Pruning은 예제코드에 적힌 방식을 사용하여 진행하였습니다. 양자화는 모델의 형식을 바꾸기 때문에, 양자화 이전에 pruning단계를 먼저 적용하였습니다. 

pruning비율은 크기 하위 5%정도만 적용하였습니다.

```python
def sparsify_onnx_weights(model, k):
    total_weights = len(model.graph.initializer)
    
    for idx, tensor in enumerate(model.graph.initializer):
        weight_array = numpy_helper.to_array(tensor)
        
        if weight_array.ndim < 2:
            continue
            
        layer_copy = weight_array.flatten()
        indices = np.abs(layer_copy).argsort()
        n_prune = int(len(indices) * k)
        indices_to_zero = indices[:n_prune]
        layer_copy[indices_to_zero] = 0
        pruned_weight = layer_copy.reshape(weight_array.shape)
        new_tensor = numpy_helper.from_array(pruned_weight, tensor.name)
        model.graph.initializer.remove(tensor)
        model.graph.initializer.append(new_tensor)

    print("Pruning Complete.")
    return model

    
if __name__ == "__main__":
    model = sparsify_onnx_weights(model, k=0.05)
```

## Quantization

처음에는 단순히 모델의 가중치 타입을 전환하는 것으로 양자화를 시도했습니다.

```python
def apply_fp16(model):
    return fp16_model = float16.convert_float_to_float16(model)

if __name__ == "__main__":
    model = apply_fp16(model)
```

이 코드를 적용함으로써 기존의 모델 용량인 75.38만 bytes를 38.08만 bytes 정도로 낮출 수 있었습니다. 하지만, 속도가 기존에 아무것도 감지하지 않는 경우 9fps에서 7fps로, 감지시에는 보통 6fps정도까지 느려지는 현상이 관찰되어 양자화 전략을 수정하게 되었습니다.


양자화 과정에는 TensorRT를 사용하여 모델의 weight를 float32에서 float16으로 jetson nano에 더 최적화된 데이터 타입으로 전환하는 방식을 사용하였습니다. 그 변환과정을 적용하는 코드는 아래와 같습니다.

```python

```